

#CNN1

In [1]:
!pip install torch

In [2]:
!pip install torchvision

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn as nn
import torch.nn.functional as F



In [4]:
# Loading of the MNIST dataset
train_data = datasets.MNIST(
    root = 'data',  # root: The root directory where the dataset should be stored. In this case, it is set to 'data'. If the 'data' directory doesn't exist, the dataset will be downloaded to this location.
    train = True,
    transform = transforms.ToTensor(),  # transform: This parameter applies transformations to the data. In this case, transforms.ToTensor() is used to convert the images to PyTorch tensors.
    download = True,
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = transforms.ToTensor()
)

In [5]:
#Setting of the main hyper-parameters of the model
batch_size = 4 # The number of samples in each mini-batch used during training. Smaller batch sizes can lead to faster convergence but may introduce more noise into the training process.
n_train = batch_size * 125    # The total size of the training dataset. It's calculated as the product of batch_size and the number of batches (125 in this case). Adjusting the training dataset size can impact the model's ability to generalize.
n_test = batch_size * 25     # The total size of the test dataset. Similar to n_train, it's calculated as the product of batch_size and the number of test batches (25 in this case). The test dataset is used to evaluate the model's performance on unseen data.
n_channels = 4  # The number of channels in the output of the quantum convolution layer. In your model, you have set it to 4. This parameter determines the depth of the feature maps produced by the convolutional layer.
initial_lr =  0.005     # The initial learning rate for the stochastic gradient descent (SGD) optimizer.

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Convolutional layer 1 with 1 input channels, 4 output channels, and 2x2 kernel
        self.conv = nn.Conv2d(1, 1, 4, stride=4)
        self.fc = nn.Linear(1 * 7 * 7, 10)

    def forward(self, x):
        # Propagate the input through the CNN layers
        x = self.conv(x)
        # Flatten the output from the convolutional layer
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc(x))
        return x


In [7]:
cnn = Net()
dataset  = train_data
train_size = n_train
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
for data in train_loader:
    inputs, labels = data
    print(f"{inputs.shape=}")
    print(f"{labels=}")
    outputs = cnn(inputs)
    print(f"{outputs.shape=}")
    print(f"{outputs=}")
    break

inputs.shape=torch.Size([4, 1, 28, 28])
labels=tensor([1, 1, 3, 7])
outputs.shape=torch.Size([4, 10])
outputs=tensor([[0.0000, 0.1221, 0.2777, 0.0000, 0.0000, 0.2654, 0.0000, 0.0000, 0.0000,
         0.1011],
        [0.0000, 0.1076, 0.1152, 0.0000, 0.0000, 0.0831, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0919, 0.2134, 0.0000, 0.0000, 0.1879, 0.1266, 0.0000, 0.0000,
         0.0427],
        [0.0000, 0.0870, 0.1450, 0.0000, 0.0235, 0.0715, 0.0862, 0.0000, 0.0000,
         0.0000]], grad_fn=<ReluBackward0>)


In [8]:
# Train the model

import datetime
import os

dataset  = train_data

# Initialize your QCNN model
cnn = Net()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.SGD(cnn.parameters(), lr=initial_lr, momentum=0.90)  # Stochastic Gradient Descent optimizer
# Create a learning rate scheduler
# Here, we use StepLR which reduces the learning rate by a factor every step_size epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=1.0)
# Split your data into training and validation sets
train_size = n_train
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "ImgClass-Quanvolv.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

RESUME_TRAINING = True

# Training loop
num_epochs = 20
loss_list = []
cnn.train()

if RESUME_TRAINING is False:
    print(f"Restore model state from {MODEL_SAVE_PATH}")
    if os.path.exists(MODEL_SAVE_PATH):
        model_dict = torch.load(MODEL_SAVE_PATH)
        initial_epoch = model_dict['epoch'] + 1
        cnn.load_state_dict(model_dict['model_state_dict'])
        optimizer.load_state_dict(model_dict['optimizer_state_dict'])
        loss_list = model_dict['loss'].copy()
    else:
        print(f"No saved model state found. Training from scratch.")
        initial_epoch = 0
        loss_list = []
else:
    initial_epoch = 0
    loss_list = []

for epoch in range(num_epochs):
    ct = datetime.datetime.now()
    # Decay Learning Rate
    optimizer.step()
    scheduler.step()
    lr = scheduler.get_last_lr()
    print(f"{epoch=}, {lr=}, {ct}")
    running_loss = []
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()  # Zero the parameter gradients to avoid accumulation
        outputs = cnn(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backpropagation
        running_loss.append(loss.item())
        optimizer.step()  # Update the model parameters
    loss_list.append(sum(running_loss) / len(running_loss))
    print('Training [{:.0f}%]\tLoss: {:.4f}'.format(100. * (epoch + 1) / num_epochs, loss_list[-1]))
    torch.save({
        'epoch': epoch,
        'model_state_dict': cnn.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss_list,
    }, MODEL_SAVE_PATH)
    print(f"Saving model state to {MODEL_SAVE_PATH}")

print('Finished Training')

epoch=0, lr=[0.005], 2024-10-15 20:49:45.862693
Training [5%]	Loss: 2.1741
Saving model state to models\ImgClass-Quanvolv.pth
epoch=1, lr=[0.005], 2024-10-15 20:49:46.543758
Training [10%]	Loss: 1.1545
Saving model state to models\ImgClass-Quanvolv.pth
epoch=2, lr=[0.005], 2024-10-15 20:49:47.143205
Training [15%]	Loss: 0.7738
Saving model state to models\ImgClass-Quanvolv.pth
epoch=3, lr=[0.005], 2024-10-15 20:49:47.793785
Training [20%]	Loss: 0.6857
Saving model state to models\ImgClass-Quanvolv.pth
epoch=4, lr=[0.005], 2024-10-15 20:49:48.469128
Training [25%]	Loss: 0.6340
Saving model state to models\ImgClass-Quanvolv.pth
epoch=5, lr=[0.005], 2024-10-15 20:49:49.076439
Training [30%]	Loss: 0.6254
Saving model state to models\ImgClass-Quanvolv.pth
epoch=6, lr=[0.005], 2024-10-15 20:49:49.746351
Training [35%]	Loss: 0.5701
Saving model state to models\ImgClass-Quanvolv.pth
epoch=7, lr=[0.005], 2024-10-15 20:49:50.357790
Training [40%]	Loss: 0.5654
Saving model state to models\ImgClas

In [9]:
cnn.state_dict()

OrderedDict([('conv.weight',
              tensor([[[[ 1.1446,  0.7152,  0.8819,  1.3776],
                        [ 0.6539,  0.9158,  0.7266,  0.4113],
                        [ 0.2243, -0.1708,  0.8783,  1.1280],
                        [ 1.2809,  0.9832,  0.6123,  0.7836]]]])),
             ('conv.bias', tensor([-1.4687])),
             ('fc.weight',
              tensor([[-0.0241, -0.0213,  0.1167,  0.0441, -0.1734,  0.0684, -0.1296, -0.0161,
                       -0.1210, -0.1907,  0.0518,  0.4501, -0.2800, -0.0461, -0.1487,  0.0155,
                        0.5237,  0.3762, -0.1925,  0.2203,  0.0515,  0.0814, -0.1849,  0.4397,
                       -1.3284, -0.8838,  0.4250,  0.0581, -0.0320,  0.5661,  0.5300, -0.5805,
                       -0.2945,  0.1406,  0.0403,  0.1289, -0.0665,  0.5522,  0.2200, -0.1486,
                       -0.1166,  0.0991,  0.1043,  0.1099,  0.1517,  0.0986, -0.3802, -0.0599,
                        0.0981],
                      [-0.0527,  0.0327, 

In [10]:
#accuracy

# Use a small subset of the full validation dataset
from torch.utils.data import SubsetRandomSampler

K = n_test # enter your length here
subsample_train_indices = torch.randperm(len(val_set))[:K]
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, sampler=SubsetRandomSampler(subsample_train_indices))

correct = 0
total = 0
# Set the model to evaluation mode
cnn.eval()
with torch.inference_mode():
    for data in val_loader:
        images, labels = data
        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy on the validation set: {100 * correct / total:.2f}%')

Accuracy on the validation set: 81.00%
